# Gif/Video maker for scrapped snapshots

In [2]:
import os
import cv2

snapshot_dir = 'Nenana_snapshots_part1/'


# List image files in the directory
image_files = sorted([snapshot_dir + f for f in os.listdir(snapshot_dir) if f.endswith('.jpg')])

# Check if there are any image files
if not image_files:
    print(f"Error: No image files found in '{snapshot_dir}'.")
    exit()

# Read the first image to get dimensions
first_image = cv2.imread(image_files[0])
if first_image is None:
    print(f"Error: Unable to read image file '{image_files[0]}'.")
    exit()

height, width, _ = first_image.shape

output_video = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), 120, (width, height))  # FPS and dimensions of frame

# Iterate through image files and add write the frame
for image_file in image_files:
    frame = cv2.imread(image_file)
    if frame is None:
        print(f"Error: Unable to read image file '{image_file}'. Skipping.")
        continue
    output_video.write(frame)

# Release video writer object
output_video.release()

print("Video created successfully.")



Video created successfully.


In [2]:
import cv2
import numpy as np

# Read the video
video_path = 'output.avi'
cap = cv2.VideoCapture(video_path)

# Parameters for Shi-Tomasi corner detection
feature_params = dict(maxCorners=100, qualityLevel=0.65, minDistance=10, blockSize=10)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=4, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Read the first frame
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Detect corners in the first frame
prev_points = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(prev_frame)

while True:
    # Read the next frame
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prev_points, None, **lk_params)
    
    # Select good points
    good_new = next_points[status == 1]
    good_old = prev_points[status == 1]
    
    # Draw tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)
        c, d = old.ravel().astype(int)
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
    
    # Overlay the tracks on the frame
    img = cv2.add(frame, mask)
    
    # Display the resulting frame
    cv2.imshow('Frame', img)
    
    # Update previous points and previous frame
    prev_gray = frame_gray.copy()
    prev_points = good_new.reshape(-1, 1, 2)
    
    # Exit on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


TypeError: 'NoneType' object is not subscriptable

: 

In [3]:
import cv2
import numpy as np
import time

# Function to detect melting areas in a frame and add lines
def detect_melting_areas(frame, mask, vertical_line_position, horizontal_line_position):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply mask to focus only on the lower half
    masked_gray = cv2.bitwise_and(gray, gray, mask=mask)
    
    # Apply thresholding to identify melting areas
    _, thresh = cv2.threshold(masked_gray, 150, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw contours on original frame
    melting_areas = frame.copy()
    cv2.drawContours(melting_areas, contours, -1, (255, 255, 0), 1)
    
    # Draw vertical and horizontal lines
    cv2.line(melting_areas, (vertical_line_position, 0), (vertical_line_position, frame.shape[0]), (0, 0, 255), 1)  # Vertical line
    cv2.line(melting_areas, (0, horizontal_line_position), (frame.shape[1], horizontal_line_position), (0, 0, 255), 1)  # Horizontal line
    
    return melting_areas

# Load the video
cap = cv2.VideoCapture('output.avi')

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create VideoWriter object to save the processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Choose the codec (codec might vary based on the operating system)
out = cv2.VideoWriter('output_with_traces.mp4', fourcc, fps, (frame_width, frame_height))

# Create a mask to focus only on the lower half of the frame
mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
mask[frame_height//2:,:] = 255

# Set desired FPS for processing
desired_fps = 900

# Calculate the time interval between frames for the desired FPS
frame_interval = 1 / desired_fps

# Define the location of the vertical and horizontal lines
vertical_line_position = 610  # 610 pixels from the left
horizontal_line_position = 592  # 592 pixels from the top

while cap.isOpened():
    # Read a frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect melting areas and add lines in the current frame
    melting_areas_frame = detect_melting_areas(frame, mask, vertical_line_position, horizontal_line_position)
    
    # Write the frame with processing traces to the output video
    out.write(melting_areas_frame)
    
    # Display the result
    cv2.imshow('Melting Areas', melting_areas_frame)
    
    # Wait for the calculated time interval between frames
    time.sleep(frame_interval)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
out.release()
cv2.destroyAllWindows()

